In [1]:
# python 2.7 with anaconda

import numpy as npmovies
from sklearn import datasets
from scipy.stats import mode
import scipy as sp
import pandas as pd

In [2]:
# Data set: Movie Lens 20M (http://grouplens.org/datasets/movielens/20m/)

genome_scores = pd.read_csv('ml-20m/genome-scores.csv')
genome_tags = pd.read_csv('ml-20m/genome-tags.csv')
links = pd.read_csv('ml-20m/links.csv')
movies = pd.read_csv('ml-20m/movies.csv')
ratings = pd.read_csv('ml-20m/ratings.csv')
tags = pd.read_csv('ml-20m/tags.csv')

In [21]:
# print(movies.info())

In [3]:
for ds in (genome_scores, genome_tags, links, movies, ratings, tags):
    display(ds.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11709768 entries, 0 to 11709767
Data columns (total 3 columns):
movieId      int64
tagId        int64
relevance    float64
dtypes: float64(1), int64(2)
memory usage: 268.0 MB


None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1128 entries, 0 to 1127
Data columns (total 2 columns):
tagId    1128 non-null int64
tag      1128 non-null object
dtypes: int64(1), object(1)
memory usage: 17.7+ KB


None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27278 entries, 0 to 27277
Data columns (total 3 columns):
movieId    27278 non-null int64
imdbId     27278 non-null int64
tmdbId     27026 non-null float64
dtypes: float64(1), int64(2)
memory usage: 639.4 KB


...

None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000263 entries, 0 to 20000262
Data columns (total 4 columns):
userId       int64
movieId      int64
rating       float64
timestamp    int64
dtypes: float64(1), int64(3)
memory usage: 610.4 MB


None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 465564 entries, 0 to 465563
Data columns (total 4 columns):
userId       465564 non-null int64
movieId      465564 non-null int64
tag          465548 non-null object
timestamp    465564 non-null int64
dtypes: int64(3), object(1)
memory usage: 14.2+ MB


None

In [4]:
def add_genres(movies):
    new_movies = movies.copy()
    
    for ix in range(0, len(movies)):
        genres = movies['genres'][ix]
        new_movies.at[ix, 'genres'] = [] if genres == "(no genres listed)" \
                                         else genres.split('|')

    return new_movies

movies_with_genres = add_genres(movies.head())
# display(movies_with_genres)

In [5]:
movies = movies_with_genres
display (ratings['rating'].min()) # 0.5 star - 5.0 stars

0.5

In [6]:
user_rating = ratings.head(1000000)\
              .pivot(index = 'userId', columns ='movieId', values = 'rating')\
              .fillna(0)

In [29]:
# display (user_rating.tail(10))
# display (ratings.tail(10))

In [7]:
print(user_rating.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6743 entries, 1 to 6743
Columns: 13950 entries, 1 to 130642
dtypes: float64(13950)
memory usage: 717.7 MB
None


In [70]:
import numpy as np
import scipy as sp

def pearson(x, y):
    if len(x.shape) == 1:
        x = x.reshape((1, -1))
    if len(y.shape) == 1:
        y = y.reshape((1, -1))
        
    prsn = np.zeros((x.shape[0], y.shape[0]))
    for xi in np.arange(x.shape[0]):
        for yi in np.arange(y.shape[0]):
            prsn[xi, yi] = sp.stats.pearsonr(x[xi,:], y[yi,:])[0]
    
    return prsn

x = user_rating.head(10)
a = pearson(np.array(x[:2]), np.array(x[:5]))
print(a)

[[ 1.          0.09659002  0.25177142  0.02438785  0.13284701]
 [ 0.09659002  1.          0.17436722  0.06139905  0.13633738]]
